In [1]:
# Jupyter
from IPython.display import display, clear_output
import ipywidgets as widgets

# Python API
from arcgis.geometry import Geometry, SpatialReference
from arcgis.gis import GIS

# NGA
from src.uid_evaluation import eval_service_attributes, eval_service_geometries
from osm_runner import *

# Other
import getpass
import time

In [ ]:
# Get Target GIS for Item Publication
gis = GIS()
while gis._url == 'http://www.arcgis.com':
    
    clear_output()
    
    gis_url  = input('Please Enter Your GIS URL:  ')
    username = input('Please Enter Your Username: ')
    password = getpass.getpass('Please Enter Your Password: ')
    
    try:
        gis = GIS(gis_url, username, password)
                  
    except:
        pass
    
print('Connected: {}'.format(gis))

In [ ]:
# Set OSM Tags with Friendly Names
osm_tag_dict = {
    "Aerialway":        "aerialway",
    "Aeroway":          "aeroway",
    "Amenity":          "amenity",
    "Barrier":          "barrier",
    "Boundary":         "boundary",
    "Building":         "building",
    "Craft":            "craft",
    "Emergency":        "emergency",
    "Geological":       "geological",
    "Highway":          "highway",
    "Historic":         "historic",
    "Landuse":          "landuse",
    "Leisure":          "leisure",
    "Man Made":         "man_made",
    "Military":         "military",
    "Natural":          "natural",
    "Office":           "office",
    "Place":            "place",
    "Power":            "power",
    "Public Transport": "public transport",
    "Railway":          "railway",
    "Route":            "route",
    "Shop":             "shop",
    "Sport":            "sport",
    "Tourism":          "tourism",
    "Waterway":         "waterway"
}

In [ ]:
%%html
<style>
.intro {
    padding: 10px; 
    color: #202020;
    font-family: 'Helvetica'
}
.map {
    border: solid;
    height: 450px;
}
</style>

## GEOINT Dashboard

### Aftering logging in to your Portal, you can use the console below to query OSM data and compare two time periods against the GEOINT tools. Selecting the Run GEOINT tool will create outputs for the OSM features at time one and two, as well as any outputs from the Attributes and Geometry GEOINT analysis tool.

In [ ]:
# Converting Map Widget Extent into a Tuple for OSM Query
def collect_extent(e):

    # Strip Min/Max For Geometry Iterable
    min_set = {k[:1]:v for k,v in e.items() if k in ['ymin', 'xmin']}
    max_set = {k[:1]:v for k,v in e.items() if k in ['ymax', 'xmax']}

    box = []
    for geo_set in [min_set, max_set]:
        # Create Arbitrary Point Geometry For Values in Decimal Degrees
        geo_set.update({'spatialReference': e.get('spatialReference')})
        point = Geometry(geo_set).project_as(4326)
        box.append(point.get('y'))
        box.append(point.get('x'))   

    return tuple(box)

In [ ]:
# on_click() Logic
def running(button_object):
    
    clear_output()
    
    # Get Date YYYY-MM-DD From DatePicker
    t1  = str(t1_sel.value)[:10] if t1_sel.value else None
    t2  = str(t2_sel.value)[:10] if t2_sel.value else None

    # Collect OSM Values
    geo_val = geo_sel.value
    osm_val = osm_sel.value
    bbox    = collect_extent(viz_map.extent)

    if not t1 or not t2:
        print('Please Specify A Start & End Time')
        
    else:
        print('Creating OSM SDF for Time One')
        t1_sdf = gen_osm_sdf(geo_val, bbox, osm_val, t1)
        print('Features Returned: {}'.format(len(t1_sdf)))
        
        t1_color = [46, 255, 0, 255]
        map_draw(t1_sdf.to_featureset(), t1_color)
        
        print('Creating OSM SDF for Time Two')
        t2_sdf = gen_osm_sdf(geo_val, bbox, osm_val, t2)
        print('Features Returned: {}'.format(len(t2_sdf)))
        
        t2_color = [0, 42, 255, 255]
        map_draw(t2_sdf.to_featureset(), t2_color)
        
        print('Creating Hosted Feature Layer for Time One . . .')
        t1_lyr = t1_sdf.to_featurelayer(
            '{}_{}'.format('Time One', int(time.time())),
            gis=gis, 
            tags='GEOINT'
        )
        display(t1_lyr)
        
        print('Creating Hosted Feature Layer for Time Two . . .')
        t2_lyr = t2_sdf.to_featurelayer(
            '{}_{}'.format('Time Two', int(time.time())),
            gis=gis, 
            tags='GEOINT'
        )
        display(t2_lyr)
        
        # Process GEOINT
        if tool_sel.value == 'Attributes':
            print('Processing Attributes with GEOINT . . .')
            res_list = (eval_service_attributes(t1_lyr.url, t2_lyr.url, 'osm_id', gis=gis))
            for item in res_list:
                display(item)
                
        else:
            print('Processing Geometries with GEOINT . . .')
            res_list = (eval_service_geometries(t1_lyr.url, t2_lyr.url, 'osm_id', gis=gis))
            for item in res_list:
                display(item)

In [ ]:
# Create Popup
def map_draw(feature_set, color):
    
    for feature in feature_set:
        viz_map.draw(
            feature.geometry,
            popup={
                'title': 'OSM ID: ' + feature.attributes['osm_id'] , 
                'content': "{}".format(
                    '<br/>'.join([
                        '%s: %s' % (key.upper(), value) for (key, value) in feature.attributes.items()
                    ])
                )
            },
            symbol={'type': 'esriSLS', 'color': color}
        )

In [ ]:
# Create & Display Map
viz_map = gis.map('Stanton Park, DC')
viz_map.basemap = 'dark-gray'
display(viz_map)

# Set Options For Return Geometry
tool_sel = widgets.Dropdown(
    options=['Attributes', 'Geometries'],
    description='Run Selected Tool',
    value='Attributes'
)

# Set Options For Return Geometry
geo_sel = widgets.Dropdown(
    options=['Point', 'Line', 'Polygon'],
    description='Geometry',
    value='Line'
)

# Set Options for OSM Tags
osm_sel = widgets.Dropdown(
    options=(sorted(osm_tag_dict.items(), key=lambda item: item[0])),
    description='Feature',
    value='highway'
)

# Create Submit Button & Set on_click
run_btn = widgets.Button(
    description='Run GEOINT',
    button_style='success',
    tooltip='Run Selected GEOINT Tool',
    layout=widgets.Layout(justify_content='center', margin='0px 0px 0px 10px')
)
run_btn.on_click(running)

# Set Options for Time Selection
t1_sel = widgets.DatePicker(description='Time One')
t2_sel = widgets.DatePicker(description='Time Two')

# Handle Widget Layout
params_a = widgets.HBox(
    [geo_sel, osm_sel, t1_sel, t2_sel], 
    layout=widgets.Layout(justify_content='center', margin='10px')
)
# Handle Widget Layout
params_b = widgets.HBox(
    [tool_sel, run_btn], 
    layout=widgets.Layout(justify_content='center', margin='10px')
)
display(params_a)
display(params_b)